### Spam Ham classification using Decision Trees

#### 1. Data Preprocessing
Means, we have to load the data, look into it, and perform the preprocessing (cleaning) if requried.

In [142]:
# importing  libraries to do EDA
import pandas as pd
import os
import numpy as np

reading the dataset and storing as pandas dataframe

In [143]:
df = pd.read_csv(os.path.join(os.getcwd(), "datasets", "spamhamdata.csv"), encoding="utf-8")

In [144]:
# let's see couple of rows in dataset
df.head(2)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
df.columns

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22',
       'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26',
       'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37'],
      dtype='object')

so many columns are showing here, basically those might be because of split of the value in csv, 
so we will make only two columns first for sms_category (spam/ham) and another one as sms_text

In [146]:
# we know v1 is the category so will rename that
df["v1"].value_counts()

v1
ham     4825
spam     747
Name: count, dtype: int64

In [ ]:
df.rename(columns={"v1": "sms_category", "v2":"sms_text"}, inplace=True) # inplace=True will make the changes in df
df.columns 

Index(['email_category', 'email_text', 'Unnamed: 2', 'Unnamed: 3',
       'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
       'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28',
       'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36',
       'Unnamed: 37'],
      dtype='object')

okay, so we saw we had NaN as a value for most of the unwanted columns, so let's remove those and concat with sms_text column

In [148]:
# replacing the np.nan with nothing and apllyig the changes 
df.replace(np.nan, '', inplace=True)
df.head(2)

,email_category,email_text,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37
0,ham,"Go until jurong point, crazy.. Available only ...",,,,,,,,,...,,,,,,,,,,
1,ham,Ok lar... Joking wif u oni...,,,,,,,,,...,,,,,,,,,,


In [149]:
# let's cpmbined the values 
# so iloc[:,:]; first colon is for rows, and second colon is for column selection
df["combined_values"] = df.iloc[:,1:].apply(lambda x: ' '.join(x.astype(str) ), axis=1)

so we got the combined text, now will retain only two columns from df

In [ ]:
df = df[["sms_category", "combined_values"]].rename(columns={"combined_values":"sms_text"}).reset_index(drop=True)

In [151]:
df.head(2)

,email_category,email_text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni... ...


superb! now let's clean the sms_text by removing punctuations and stop words, etc. 

In [152]:
# as applymap has deprecated, will use map; it will take each element and does the transformation
df = df.map(lambda x:  x.lower()) 
df.head(2)

,email_category,email_text
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni... ...


In [153]:
# first will remove the punctuations
import string

In [154]:
punctuations = list(string.punctuation)

def remove_punctuations(x:str):
    x = list(x)
    for p in punctuations:
        try:
            x.remove(p)
        except Exception as _:
            pass

    return ' '.join(''.join(x).split()) # removing any whitespaces 

In [ ]:
# here we are calling the method directly upon the column, thus we do not need to specify the axis
df["sms_text"] = df["sms_text"].apply(remove_punctuations) 
df.head(2)

,email_category,email_text
0,ham,go until jurong point crazy. available only in...
1,ham,ok lar.. joking wif u oni...


let's remove the stop words, as they do not add weight into the sentence, so to do so we will use nltk to remove the stop words

In [157]:
import nltk
from nltk.corpus import stopwords

In [158]:
# before using it we have to download that first
# nltk.download('stopwords')

In [ ]:
# custom method to remove stopwords from the sms text
_stopwords = stopwords.words("english")
def remove_stopwords(x:str):
    return ' '.join([item for item in x.split() if item not in _stopwords])

In [ ]:
df["sms_text"] = df["sms_text"].apply(remove_stopwords)
df.head(2)

,email_category,email_text
0,ham,go jurong point crazy. available bugis n great...
1,ham,ok lar.. joking wif u oni...


Nice! we have did basic preprocessing so far, and should be good to proceed further

#### 2. EDA - Exploratory Data Analysis
Means, we will understand the data by analyzing the relationship between features, importance of features etc.

In [ ]:
# so this is very small and simple dataset, but will see what we can analyze here
df["sms_category"].value_counts()

email_category
ham     4825
spam     747
Name: count, dtype: int64

No!!! given data set is very imbalance data set. If we train the model with this dataset, then definetly it would be biased towards ham

In [ ]:
# SMOTE
# imbalances- learn